In [5]:
#CRNN
#Edit:2017-09-14 ~ 2017-09-17
#@sima
#%%
from keras.layers.convolutional import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Reshape,Masking,Lambda,Permute
from keras.layers import Input,Dense,Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.layers.recurrent import GRU,LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import Model
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import Adam,SGD,Adadelta
from keras import losses
from keras.layers.wrappers import TimeDistributed
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard
from keras.utils import plot_model
from matplotlib import pyplot as plt

import numpy as np 
import os
from PIL import Image 
import json
import threading

import tensorflow as tf  
import keras.backend.tensorflow_backend as K  

def get_session(gpu_fraction=0.6):  
    '''''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''  
  
    num_threads = os.environ.get('OMP_NUM_THREADS')  
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)  
  
    if num_threads:  
        return tf.Session(config=tf.ConfigProto(  
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))  
    else:  
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))  
  
K.set_session(get_session()) 



def ctc_lambda_func(args):
    y_pred,labels,input_length,label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


char=''
with open('E:\deeplearn\OCR\chi_sim\随机语料\yu\char.txt',encoding='utf-8') as f:
      for ch in f.readlines():
            ch = ch.strip('\r\n')
            char=char+ch
char =char+'^'
print('nclass:',len(char))

char_to_id = {j:i for i,j in enumerate(char)}
id_to_char = {i:j for i,j in enumerate(char)}

maxlabellength = 20
img_h = 32
img_w = 248
nclass = len(char)
rnnunit=256
batch_size =64
#gen = image.ImageDataGenerator(rescale=1.0/255)


def gen1(jsonpath,batchsize=64,maxlabellength=8,imagesize=(32,230)):
    with open(jsonpath,'r',encoding='utf-8') as f:
        image_label = json.load(f)
    imagepathlist = [i for i,_ in image_label]
    imagepathlist = np.array(imagepathlist)
    while 1:
       
       labels = np.ones([batchsize,maxlabellength])
       input_length = np.zeros([batchsize,1])
       label_length = np.zeros([batchsize,1])


def gen2(jsonpath,imagepath,batchsize=64,maxlabellength=8,imagesize=(32,248)):
    with open(jsonpath,'r',encoding='utf-8') as f:
        image_label = json.load(f)

    print('open json')
    imagelabel =[i['label'] for _,i in image_label.items()]
    _imagefile = [i for i,j in image_label.items()]
    print('--begin gen2')
    v = gen.flow_from_directory(imagepath,target_size=imagesize,
                            color_mode='grayscale',class_mode='sparse',shuffle=True,
                            #save_to_dir=r'E:\deeplearn\OCR\Sample\fixsizetrain',
                            batch_size = batchsize
                            )

    v.classes = np.array([i for i in range(len(imagelabel))])
    v.filenames = _imagefile
    print('end gen2')
    
    while 1:
       x,l = next(v)
       bz = len(l)
       labels = np.ones([bz,maxlabellength])
       input_length = np.zeros([bz,1])
       label_length = np.zeros([bz,1])
       for i in range(bz):
           str = imagelabel[l[i]]
           label_length[i] = len(str)        

           input_length[i] = imagesize[1]//4+1
           labels[i,:len(str)] =[char_to_id[i] for i in str]
#            print(str)
#        print(labels)
#        print(_imagefile[l[i]])
#        print(label_length)
#        print(input_length)

       inputs = {'the_input': x,
                'the_labels': labels,
                'input_length': input_length,
                'label_length': label_length,
                }
       outputs = {'ctc': np.zeros([batchsize])} 
       #output = [x,labels,input_length,label_length]
       yield (inputs,outputs)
        
class random_uniform_num():
    """
    均匀随机，确保每轮每个只出现一次
    """
    def __init__(self,total):
        self.total = total
        self.range = [i for i in range(total)]
        np.random.shuffle(self.range)
        self.index = 0
    def get(self,batchsize):
        r_n=[]
        if(self.index+batchsize>self.total):
            r_n_1 = self.range[self.index:self.total]
            np.random.shuffle(self.range)
            self.index = (self.index+batchsize)-self.total
            r_n_2 = self.range[0:self.index]
            r_n.extend(r_n_1)
            r_n.extend(r_n_2)
            
        else:
            r_n = self.range[self.index:self.index+batchsize]
            self.index = self.index+batchsize
        return r_n    
    
def gen3(jsonpath,imagepath,batchsize=64,maxlabellength=8,imagesize=(32,356)):
    with open(jsonpath,'r',encoding='utf-8') as f:
        image_label = json.load(f)

    print('open json')
    #imagelabel =[i['label'] for _,i in image_label.items()]
    _imagefile = [i for i,j in image_label.items()]
    x = np.zeros((batchsize, imagesize[0], imagesize[1], 1), dtype=np.float)
    labels = np.ones([batchsize,maxlabellength])*10000
    input_length = np.zeros([batchsize,1])
    label_length = np.zeros([batchsize,1])
    
    r_n = random_uniform_num(len(_imagefile))
    print('图片总量',len(_imagefile))
    _imagefile = np.array(_imagefile)
   
    while 1:
       
       shufimagefile = _imagefile[r_n.get(batchsize)]
       for i,j in enumerate(shufimagefile):
           img1 = Image.open(j)
           img = np.array(img1,'f')/255.0-0.5
           
           x[i] = np.expand_dims(img,axis=2)
           #print('imag:shape',img.shape)
           str = image_label[j]['label']
           label_length[i] = len(str)        
           if(len(str)<=0):
                print("len<0",j)
           input_length[i] = imagesize[1]//4+1
           labels[i,:len(str)] =[char_to_id[i] for i in str]
        
#        print('-------------------------------')
#        print('x shape:',x.shape)
#        print(str)
#        print('labes',labels)
#        print('labellength:',label_length)
#        print('inputlenght',input_length)
#        print(x)

       inputs = {'the_input': x,
                'the_labels': labels,
                'input_length': input_length,
                'label_length': label_length,
                }
       outputs = {'ctc': np.zeros([batchsize])} 
       yield (inputs,outputs)
        

input = Input(shape=(img_h,None,1),name='the_input')
m = Conv2D(64,kernel_size=(3,3),activation='relu',padding='same',name='conv1')(input)
m = MaxPooling2D(pool_size=(2,2),strides=(2,2),name='pool1')(m)
m = Conv2D(128,kernel_size=(3,3),activation='relu',padding='same',name='conv2')(m)
m = MaxPooling2D(pool_size=(2,2),strides=(2,2),name='pool2')(m)
m = Conv2D(256,kernel_size=(3,3),activation='relu',padding='same',name='conv3')(m)
m = BatchNormalization(axis=3)(m)
m = Conv2D(256,kernel_size=(3,3),activation='relu',padding='same',name='conv4')(m)

m = ZeroPadding2D(padding=(0,1))(m)
m = MaxPooling2D(pool_size=(2,2),strides=(2,1),padding='valid',name='pool3')(m)

m = Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='conv5')(m)
m = BatchNormalization(axis=3)(m)
m = Conv2D(512,kernel_size=(3,3),activation='relu',padding='same',name='conv6')(m)

m = ZeroPadding2D(padding=(0,1))(m)
m = MaxPooling2D(pool_size=(2,2),strides=(2,1),padding='valid',name='pool4')(m)
m = Conv2D(512,kernel_size=(2,2),activation='relu',padding='valid',name='conv7')(m)
m = BatchNormalization(axis=3)(m)

m = Permute((2,1,3),name='permute')(m)
m = TimeDistributed(Flatten(),name='timedistrib')(m)

m = Bidirectional(GRU(rnnunit,return_sequences=True,implementation=2),name='blstm1')(m)
#m = Bidirectional(LSTM(rnnunit,return_sequences=True),name='blstm1')(m)
m = Dense(rnnunit,name='blstm1_out',activation='linear',)(m)
#m = Bidirectional(LSTM(rnnunit,return_sequences=True),name='blstm2')(m)
m = Bidirectional(GRU(rnnunit,return_sequences=True,implementation=2),name='blstm2')(m)
y_pred = Dense(nclass,name='blstm2_out',activation='softmax')(m)

basemodel = Model(inputs=input,outputs=y_pred)
basemodel.summary()


labels = Input(name='the_labels',shape=[maxlabellength],dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length]) 

model = Model(inputs=[input, labels, input_length, label_length], outputs=loss_out)


adam = Adam()
#tf_adam = tf.train.AdamOptimizer ()
#adadelta = Adadelta()
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam,metrics=['accuracy'])


checkpoint = ModelCheckpoint(r'E:\deeplearn\OCR\Sample\model\weights-{epoch:02d}.hdf5',
                           save_weights_only=True)
earlystop = EarlyStopping(patience=10)
tensorboard = TensorBoard(r'E:\deeplearn\OCR\Sample\model\tflog',write_graph=True)
                           

nclass: 3862
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 32, 100, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 100, 64)       640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 16, 50, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 50, 128)       73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 8, 25, 128)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 8, 25, 256)        295168    
_________________________________________________________________
batch_normalization_4 (Batch (None, 8, 25, 256)        1024    

In [2]:


print('-----------beginfit--')
cc1=gen3(r'E:\deeplearn\OCR\Sample\trainnew\trainlabel.json',r'E:\deeplearn\OCR\Sample\trainnew',batchsize=batch_size,maxlabellength=maxlabellength,imagesize=(img_h,img_w))
cc2=gen3(r'E:\deeplearn\OCR\Sample\validnew\validlabel.json',r'E:\deeplearn\OCR\Sample\validnew',batchsize=batch_size,maxlabellength=maxlabellength,imagesize=(img_h,img_w))


-----------beginfit--


In [4]:
#字内不上下浮动+扭曲+60颜色
model.load_weights(r'E:\deeplearn\OCR\Sample\model\weights-51.hdf5')
res = model.fit_generator(cc1,
                    steps_per_epoch =145750// batch_size,
                    epochs = 100,
                    validation_data =cc2 ,
                    validation_steps = 1003 // batch_size,
                    callbacks =[earlystop,checkpoint,tensorboard],
                    verbose=1
                    )

Epoch 1/100
open json
图片总量 145750
2276/2277 [============================>.] - ETA: 0s - loss: 0.2515 - acc: 0.9315open json
图片总量 1003
2277/2277 [==============================] - 1188s - loss: 0.2515 - acc: 0.9315 - val_loss: 0.6258 - val_acc: 0.9292
Epoch 2/100
2277/2277 [==============================] - 1197s - loss: 0.1884 - acc: 0.9425 - val_loss: 0.4036 - val_acc: 0.9437
Epoch 3/100
2277/2277 [==============================] - 1187s - loss: 0.1764 - acc: 0.9450 - val_loss: 0.5295 - val_acc: 0.9333
Epoch 4/100
2277/2277 [==============================] - 1189s - loss: 0.1387 - acc: 0.9548 - val_loss: 0.5246 - val_acc: 0.9083
Epoch 5/100
2277/2277 [==============================] - 1193s - loss: 0.1543 - acc: 0.9502 - val_loss: 0.5483 - val_acc: 0.9458
Epoch 6/100
2277/2277 [==============================] - 1198s - loss: 0.1156 - acc: 0.9616 - val_loss: 0.4577 - val_acc: 0.9354
Epoch 7/100
2277/2277 [==============================] - 1187s - loss: 0.1364 - acc: 0.9550 - val_loss:

KeyboardInterrupt: 

In [ ]:
#字内不上下浮动
res = model.fit_generator(cc1,
                    steps_per_epoch =145750// batch_size,
                    epochs = 100,
                    validation_data =cc2 ,
                    validation_steps = 1003 // batch_size,
                    callbacks =[earlystop,checkpoint,tensorboard],
                    verbose=1
                    )


Epoch 1/100
open json
图片总量 145750
2276/2277 [============================>.] - ETA: 0s - loss: 42.2532 - acc: 0.0000e+00open json
图片总量 1003
2277/2277 [==============================] - 1192s - loss: 42.2516 - acc: 0.0000e+00 - val_loss: 39.5801 - val_acc: 0.0000e+00
Epoch 2/100
2277/2277 [==============================] - 1190s - loss: 33.8430 - acc: 0.0089 - val_loss: 32.0627 - val_acc: 0.0250
Epoch 3/100
2277/2277 [==============================] - 1190s - loss: 24.9440 - acc: 0.1228 - val_loss: 26.5708 - val_acc: 0.0865
Epoch 4/100
2277/2277 [==============================] - 1190s - loss: 20.6413 - acc: 0.1948 - val_loss: 20.0942 - val_acc: 0.1781
Epoch 5/100
2277/2277 [==============================] - 1190s - loss: 16.7674 - acc: 0.2200 - val_loss: 15.5865 - val_acc: 0.2010
Epoch 6/100
2277/2277 [==============================] - 1189s - loss: 12.7008 - acc: 0.2290 - val_loss: 12.2085 - val_acc: 0.1760
Epoch 7/100
2277/2277 [==============================] - 1189s - loss: 9.0045 

In [ ]:
with open(r'E:\deeplearn\OCR\Sample\trainnew\his', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [5]:

res = model.fit_generator(cc1,
                    steps_per_epoch =145750// batch_size,
                    epochs = 100,
                    validation_data =cc2 ,
                    validation_steps = 1003 // batch_size,
                    callbacks =[earlystop,checkpoint,tensorboard],
                    verbose=1,
                    initial_epoch=9
                    )


Epoch 10/100
2276/2277 [============================>.] - ETA: 0s - loss: 6.1203 - acc: 0.2081open json
图片总量 1003
2277/2277 [==============================] - 1175s - loss: 6.1216 - acc: 0.2081 - val_loss: 19.0974 - val_acc: 0.0979
Epoch 11/100
2277/2277 [==============================] - 1175s - loss: 2.0049 - acc: 0.3155 - val_loss: 1.1752 - val_acc: 0.4854
Epoch 12/100
2277/2277 [==============================] - 1174s - loss: 0.6835 - acc: 0.6122 - val_loss: 0.8320 - val_acc: 0.7469
Epoch 13/100
2277/2277 [==============================] - 1173s - loss: 0.5192 - acc: 0.7387 - val_loss: 0.9210 - val_acc: 0.7635
Epoch 14/100
2277/2277 [==============================] - 1174s - loss: 0.3913 - acc: 0.8185 - val_loss: 0.6165 - val_acc: 0.8625
Epoch 15/100
2277/2277 [==============================] - 1174s - loss: 0.2878 - acc: 0.8751 - val_loss: 0.6487 - val_acc: 0.8771
Epoch 16/100
2277/2277 [==============================] - 1175s - loss: 0.2430 - acc: 0.8975 - val_loss: 0.7223 - val_

In [ ]:
#字内上下浮动
res = model.fit_generator(cc1,
                    steps_per_epoch =145750// batch_size,
                    epochs = 100,
                    validation_data =cc2 ,
                    validation_steps = 1003 // batch_size,
                    callbacks =[earlystop,checkpoint],
                    verbose=1,
                    initial_epoch=46
                    )

Epoch 47/100
2277/2277 [==============================] - 1178s - loss: 0.0607 - acc: 0.9788 - val_loss: 0.3766 - val_acc: 0.9271
Epoch 48/100
2277/2277 [==============================] - 1180s - loss: 0.0847 - acc: 0.9716 - val_loss: 0.5732 - val_acc: 0.9531
Epoch 49/100
2277/2277 [==============================] - 1179s - loss: 0.0557 - acc: 0.9804 - val_loss: 0.6628 - val_acc: 0.9646
Epoch 50/100
2277/2277 [==============================] - 1180s - loss: 0.0543 - acc: 0.9808 - val_loss: 0.3830 - val_acc: 0.9542
Epoch 51/100
2277/2277 [==============================] - 1182s - loss: 0.0540 - acc: 0.9806 - val_loss: 0.6029 - val_acc: 0.9469
Epoch 52/100
2277/2277 [==============================] - 1179s - loss: 0.0574 - acc: 0.9801 - val_loss: 0.5402 - val_acc: 0.8781
Epoch 53/100
2277/2277 [==============================] - 1178s - loss: 0.0658 - acc: 0.9773 - val_loss: 0.2857 - val_acc: 0.9656
Epoch 54/100
2277/2277 [==============================] - 1178s - loss: 0.0443 - acc: 0.98

In [ ]:
model.optimizer.lr =0.000000000001
model.fit_generator(cc1,
                    steps_per_epoch =36000// batch_size,
                    epochs = 3,
                    validation_data =cc2 ,
                    validation_steps = 2000 // batch_size,
                    callbacks =[tensorboard,earlystop,checkpoint],
                    verbose=1,
                    initial_epoch=2
                    )

Epoch 3/3
 128/1125 [==>...........................] - ETA: 631s - loss: 4.8391 - acc: 0.1152